In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

from PIL import Image
import numpy as np

from tqdm import trange
import random

import scipy.io
import tqdm


# Image Display

In [ ]:
def show_top_images ( dataset_path, indices , id_test , ids , labels ) :
    
    label = (ids[id_test] - 1) // 80

    name = dataset_path + '/jpg/' + str(label) + '/image_' + str(ids[id_test]).zfill(4) + '.jpg'
    image = Image.open( name )
    
    top = 0
    show_image_label(top, image, labels[id_test], ids[id_test] )
    
    accuracy = 0
    
    for i in indices[0] :
        label_i = labels[i]
        name = dataset_path + '/jpg/' + str(label_i) + '/image_' + str(ids[i]).zfill(4) + '.jpg'

        image = Image.open( name )

        show_image_label(top, image, label_i, ids[i] )   
        top = top + 1
        
    
def show_image_label ( top, image, label , image_id ) :
    
    plt.figure(figsize = (5,5))
    plt.imshow(image, aspect='auto')
    plt.axis('off')
    plt.title(f'{top} - Image id {image_id} with label {label}.')
    plt.show()

# Generate descriptors

In [ ]:
def create_deep_descriptors (image, model, my_transform, my_device='cpu') :

    model_input = my_transform(image) 
    model_input = model_input.unsqueeze_(0)
    
    model = model.to(my_device)
    model_input = model_input.to(my_device)

    model.eval()
    with torch.no_grad():
        desc_deep = model(model_input).squeeze(0)
    
    return desc_deep.to('cpu')

# Data

In [ ]:
def represent_dataset( dataset_path, model, my_transform, my_device ) :

    mat = scipy.io.loadmat( dataset_path+'/datasplits.mat' )

    ids = mat['tst1'][0] #  'tst1' or 'trn1' or 'val1' 
    
    space = []
    labels = []
    
    for id in tqdm.tqdm(ids, desc='Processing test set') :

        label = (id - 1) // 80
        name = dataset_path + '/jpg/' + str(label) + '/image_' + str(id).zfill(4) + '.jpg'

        image = Image.open( name )
        
        if image is None:
            print(f'Reading image Error. Path: {name}')
            return None

        desc_deep = create_deep_descriptors(image, model, my_transform, my_device)

        space.append(desc_deep)
        labels.append(label)
        
    print( ' -> [I] Space Describing Info:\n', 
        '\nNumber of images: ', len(space), 
        '\nNumber of labels: ', len(labels),
        '\nDimension: ', len(space[0])
        )

    return space , labels 

In [ ]:
from sklearn.neighbors import NearestNeighbors

def run_test ( space , labels , dataset_path, model, my_transform, my_device, top=10 ) :
    knn = NearestNeighbors(n_neighbors=top+1).fit(space)
    
    mat = scipy.io.loadmat( dataset_path+'/datasplits.mat' )

    ids = mat['tst1'][0] #  'tst1' or 'trn1' or 'val1'
    
    accuracy_t = 0
    
    for id_test in tqdm.tqdm(ids, desc='running the test phase') :
        
        label = (id_test - 1) // 80
        name = dataset_path + '/jpg/' + str(label) + '/image_' + str(id_test).zfill(4) + '.jpg'

        image = Image.open( name )
        
        desc_deep = create_deep_descriptors(image, model, my_transform, my_device)

        indices = knn.kneighbors(desc_deep.reshape(1, -1))[1]

        labels_top = [ labels[i] for i in indices[0] ]

        accuracy = sum( np.equal(labels_top, label) )
        accuracy =( (accuracy-1)/(top) ) * 100
        accuracy_t = accuracy_t + accuracy
        
    print(f'Average accuracy in the test set: {accuracy_t/len(ids):5.2f}%')
    

# Experimental evaluation

In [ ]:
def retrieve_single_image ( space , labels , dataset_path, model, my_transform, my_device, top=10 ) :
    knn = NearestNeighbors(n_neighbors=top+1).fit(space)
    
    mat = scipy.io.loadmat( dataset_path+'/datasplits.mat' )

    ids = mat['tst1'][0] #  'trn1' or 'val1'
    
    id_test = random.randrange( len(ids) )
        
    label = (ids[id_test] - 1) // 80
    name = dataset_path + '/jpg/' + str(label) + '/image_' + str(ids[id_test]).zfill(4) + '.jpg'
    
    image = Image.open( name )
    
    if image is None:
        print(f'Reading image Error. Path: {name}')
        return None

    desc_deep = create_deep_descriptors(image, model, my_transform, my_device)
    
    distances, indices = knn.kneighbors(desc_deep.reshape(1, -1))
    
    show_top_images(dataset_path, indices, id_test, ids, labels)
    
    labels_top = [ int(labels[i]) for i in indices[0] ]
    
    accuracy = sum( np.equal( label , labels_top ) )
    accuracy =( (accuracy-1)/(top) ) * 100 
    
    print(f'Accuracy for image id {ids[id_test]}: {accuracy:5.2f}%')
    
    print(name)    
    print(f'Image: {ids[id_test]} with label {labels[id_test]}')    
    print(f'Closest image: {ids[indices[0][0]]} with distance {distances[0][0]} and label {labels[indices[0][0]]}')
    print('Distances: ',distances)
    print('Indices: ',indices[0])
    print('Labels: ',labels_top)

# Create the Descriptor model and load weights 


Aqui vocês devem:
1. Criar o modelo descritor que é uma arquitetura CNN sem a última camada da FC de classificação. Atribua a variável `model_descriptor`.
2. Criar o objeto que indica quais são as transformações necessárias para o modelo. Atribua a variável `my_transform`.
3. Definir se os modelos vão executar na CPU ou na GPU. Atribua a variável `my_device`.
4. Faça o teste com pelo menos 3 arquiteturas diferentes e reporte o resultado da função `run_test` em formato de tabela.

# Execution

In [ ]:
dataset_path = # ********* INSERT HERE THE Flowers-17 DATASET PATH ******************

# Using model descriptor, represent all images in the testing split of the dataset. 
space, labels = represent_dataset ( dataset_path , model_descriptor, my_transform, my_device )

In [ ]:
# For each image of the testing split of the dataset, use the image as the query
# for the image retrieval problem, i.e., describe the image using the model descriptor, 
# and search for the k closest descriptors of the images in the testing split dataset. 
# After that, measure the accuracy of the image retrieval by counting how many of the k
# retrieved images has the same label of the query image, and divide it by k. 
# This result will be the accuracy of the image retrieval result for that specific image. 
# Repeat it for each image of the testing split of the dataset and compute the average of 
# all calculated accuracies.

run_test( space, labels, dataset_path, model_descriptor, my_transform, my_device )

In [ ]:
# For a random image of the testing split of the dataset, use the image as the query
# for the image retrieval problem, i.e., describe the image using the model descriptor, 
# and search for the k closest descriptors of the images in the testing split dataset. 
# After that, measure the accuracy of the image retrieval by counting how many of the k
# retrieved images has the same label of the query image, and divide it by k. 
# This result will be the accuracy of the image retrieval result for that specific image.
# Also display the query image, and all k images returned by the image retrieval problem.
# For each returned image, also display its class, and the Euclidean distance between its
# descriptor and the descriptor of the query image.
# Since the query image also is in the testing split of the dataset, it is expected that 
# the first returned image is the query image itself, and the distance between the 
# descriptors to be zero.

retrieve_single_image( space, labels, dataset_path, model_descriptor, my_transform, my_device)